In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/hearthstone-minions-stats-and-popularity/hearthstone_minions.csv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hearthstone-minions-stats-and-popularity/hearthstone_minions.csv


In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

columns_to_drop = ['url', 'card_name', 'CARDTYPE', 'COLLECTIBLE', 'ELITE', 'HAS_DIAMOND_QUALITY', 'MAX_SIDEBOARD_CARDS', 
                   'DECK_RULE_MOD_DECK_SIZE', 'TAG_SCRIPT_DATA_NUM_1', 'ENCHANTMENT_BIRTH_VISUAL', 
                   'ENCHANTMENT_IDLE_VISUAL', 'FACTION', 'AttackVisualType', 'DONT_PICK_FROM_SUBSETS', 
                   'IS_BACON_POOL_MINION', 'DECK_ACTION_COST', 'COLLECTION_RELATED_CARD_DATABASE_ID', 
                   'FORGES_INTO', 'TRIGGER_VISUAL', 'MINI_SET', 'TECH_LEVEL', 'MULTIPLE_CLASSES', 
                   'COST_UNHOLY', 'COST_FROST', 'COST_BLOOD', 'TAG_SCRIPT_DATA_NUM_2', 
                   'TRANSFORMED_FROM_CARD_VISUAL_TYPE', 'DECK_RULE_COUNT_AS_COPY_OF_CARD_ID', 'DevState',
                   'PLAYER_TAG_THRESHOLD_TAG_ID', 'PLAYER_TAG_THRESHOLD_VALUE', 'ENTITY_TAG_THRESHOLD_TAG_ID', 'ENTITY_TAG_THRESHOLD_VALUE',
                   'DISPLAY_CARD_ON_MOUSEOVER', ]
df = df.drop(columns=columns_to_drop, errors='ignore')

df['times_played'] = df['times_played'].str.replace(',', '').astype(int)
df = df[df['times_played'] >= 10]

valid_counts = df.count()
columns_to_drop = valid_counts[valid_counts < 5].index
df = df.drop(columns=columns_to_drop)

df = df[(df['avg_copies'] != '-') & (df['deck_winrate'] != '-')]

df['in_perc_of_decks'] = df['in_perc_of_decks'].str.replace('%', '').astype(float) / 100
df['deck_winrate'] = df['deck_winrate'].str.replace('%', '').astype(float) / 100
df['avg_copies'] = df['avg_copies'].astype(float)

df = df.fillna(0)

df['DISCARD_CARDS'] = df['DISCARD_CARDS'].apply(lambda x: 1 if x != 0 else 0)
df['MULTI_CLASS_GROUP'] = df['MULTI_CLASS_GROUP'].apply(lambda x: 1 if x != 0 else 0)
df['CARDRACE'] = df['CARDRACE'].apply(lambda x: 1 if x != 0 else 0)

valid_counts = df.count()

print("Features")
print(valid_counts)

Features
in_perc_of_decks         2351
avg_copies               2351
deck_winrate             2351
times_played             2351
CLASS                    2351
RARITY                   2351
COST                     2351
HEALTH                   2351
ATK                      2351
BATTLECRY                2351
DISCOVER                 2351
MANATHIRST               2351
CARDRACE                 2351
TAUNT                    2351
TITAN                    2351
USE_DISCOVER_VISUALS     2351
AURA                     2351
DEATHRATTLE              2351
FREEZE                   2351
FORGE                    2351
DIVINE_SHIELD            2351
CHARGE                   2351
HAS_SIGNATURE_QUALITY    2351
MULTI_CLASS_GROUP        2351
COMBO                    2351
CANT_BE_DAMAGED          2351
OVERLOAD                 2351
REBORN                   2351
RUSH                     2351
DREDGE                   2351
SPELLPOWER               2351
CORPSE                   2351
SECRET                   2351
M

In [3]:
# 检查数据结构和基本信息
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 2351 entries, 0 to 2350
Data columns (total 63 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   in_perc_of_decks       2351 non-null   float64
 1   avg_copies             2351 non-null   float64
 2   deck_winrate           2351 non-null   float64
 3   times_played           2351 non-null   int64  
 4   CLASS                  2351 non-null   float64
 5   RARITY                 2351 non-null   float64
 6   COST                   2351 non-null   float64
 7   HEALTH                 2351 non-null   float64
 8   ATK                    2351 non-null   float64
 9   BATTLECRY              2351 non-null   float64
 10  DISCOVER               2351 non-null   float64
 11  MANATHIRST             2351 non-null   float64
 12  CARDRACE               2351 non-null   int64  
 13  TAUNT                  2351 non-null   float64
 14  TITAN                  2351 non-null   float64
 15  USE_DISCO